In [1]:
import praw
import pandas as pd

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="_9aT35rxVcBiNC17_Al5_g",
    client_secret="tyas9JGsMfhm4Jhifg86JnvpHZHx6A",
    user_agent="CryptoSentimentApp"
)

def fetch_reddit_posts(coin, num_posts):
    """
    Fetches Reddit posts related to a cryptocurrency and returns data in a Pandas DataFrame.
    
    Parameters:
    - coin (str): The cryptocurrency to search for.
    - num_posts (int): Number of posts to retrieve.

    Returns:
    - pd.DataFrame: DataFrame containing post details.
    """
    subreddit = reddit.subreddit("cryptocurrency")
    query = coin.lower()  # Convert to lowercase for better matching
    
    # Fetch posts containing the coin name
    posts_data = []
    for post in subreddit.search(query, limit=num_posts):
        posts_data.append([
            post.id,
            post.title,
            post.selftext,  # Post content
            post.score,  # Upvotes
            post.num_comments,  # Number of comments
            post.url,  # Post link
            post.created_utc  # Timestamp
        ])

    # Convert to Pandas DataFrame
    df = pd.DataFrame(posts_data, columns=["Post_ID", "Title", "Content", "Upvotes", "Comments", "URL", "Timestamp"])
    return df

# Example usage: Fetch 10 Bitcoin-related posts
df_bitcoin_100 = fetch_reddit_posts("Bitcoin", 100)
print(df_bitcoin_100.head())
df_bitcoin_100.to_csv('csv_bitcoin_100.csv')


   Post_ID                                              Title  \
0  1gqafju  Bitcoin has followed a consistent 4-year cycle...   
1  1ik2qgu  Explaining Bitcoin 12 Years Ago When It Was Wo...   
2  1h6yoqp  On February 9th 2011 Bitcoin first touched $1....   
3  1hbsf6a  This Anonymous guy received $50 worth of Bitco...   
4  1gs7eb9  Satoshi Era Bitcoin Whale moved 2,000 BTC for ...   

                        Content  Upvotes  Comments  \
0                                   3584       700   
1                                   8422       293   
2                                   7826       495   
3  Imagine hodling for 13 Years     7415       452   
4                                   5947       608   

                                    URL     Timestamp  
0  https://i.redd.it/95px1ns8in0e1.jpeg  1.731496e+09  
1   https://i.redd.it/hd0ul5cglrhe1.png  1.738955e+09  
2   https://i.redd.it/m7ll0go40y4e1.png  1.733366e+09  
3  https://i.redd.it/ogcetnacr76e1.jpeg  1.733920e+09  
4  h